In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import numpy as np
import pandas as pd
import swifter

pd.set_option("display.max_columns", None)
# from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

plt.rcParams["lines.linewidth"] = 2
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "Helvetica"
plt.rcParams["text.usetex"] = True
plt.rcParams["font.size"] = 16
plt.rcParams["axes.spines.right"] = False
plt.rcParams["axes.spines.top"] = False
plt.rcParams["figure.dpi"] = 150
plt.rcParams["savefig.bbox"] = "tight"
from tqdm.auto import tqdm

import glob
from pathlib import Path

import h3.api.numpy_int as h3
from stc_unicef_cpi.data import process_geotiff as pg


In [ ]:
nga_df = pd.read_csv("/Users/johnf/Downloads/raw_low_res_dssg/dhs/clean_nga_dhs.csv")
# nga_df = nga_df.groupby(by=["hex_code"],as_index=False).mean()

In [ ]:
nga_df[['LATNUM','LONGNUM']].nunique()
# nga_df['cluster'].nunique()

In [ ]:
nga_df['nbr_hex']=nga_df.hex_code.swifter.apply(lambda h: h3.hex_ring(h, 1))

In [ ]:
# NB location is rural / urban
nga_df.head()

In [ ]:
(nga_df.join(nga_df
            .explode("nbr_hex")
            .set_index("nbr_hex")['hex_code'], 
            on="hex_code", 
            how="left",
            rsuffix="_nbr")
 .dropna(subset=['hex_code_nbr'])
 .groupby(by=['hex_code'])
 .hex_code_nbr.agg(lambda x: list(x))
 )


In [ ]:
new_df = pg.agg_tif_to_df(nga_df,"/Users/johnf/Downloads/raw_low_res_dssg")

# Reduce full dataset to NGA

In [ ]:
df = pd.read_csv(
    "/Users/johnf/Downloads/raw_low_res_dssg/dhs/childpoverty_microdata_gps_21jun22.csv"
)

In [ ]:
nga_df = df[df["countrycode"].str.strip() == "NGA"]


In [ ]:
nga_df.dropna(subset=["LATNUM", "LONGNUM"], inplace=True)


In [ ]:
nga_df["hex_code"] = nga_df[["LATNUM", "LONGNUM"]].apply(
    lambda row: h3.geo_to_h3(row["LATNUM"], row["LONGNUM"], 7), axis=1
)


In [ ]:
nga_df = nga_df.reset_index().drop(columns=["index"]).copy()


In [ ]:
nga_df.to_csv(
    "/Users/johnf/Downloads/raw_low_res_dssg/dhs/clean_nga_dhs.csv", index=False
)


# Load preprocessed NGA data

In [ ]:
nga_df = pd.read_csv("/Users/johnf/Downloads/raw_low_res_dssg/dhs/clean_nga_dhs.csv")


In [ ]:
nga_df.hex_code.value_counts().hist(
    bins=50,
)
plt.show()


In [ ]:
# show hists for each target value
nga_df[[col for col in nga_df.columns if "_sev" in col]].hist()
plt.show()


In [ ]:
# Show hist of missing target values
nga_df[[col for col in nga_df.columns if "_sev" in col]].isna().astype(int).hist()
plt.show()


In [ ]:
nga_df.dropna(subset=["dep_health_sev"]).hex_code.nunique()


In [ ]:
# NB this aggregates as mean for each hex code, though think 
# FB may have actually used initially mean of each cluster, 
# then mean of these clusters within each cell - may be 
# some differences
nga_df = nga_df.groupby(by=["hex_code"], as_index=False).mean()


In [ ]:
new_df = pg.agg_tif_to_df(nga_df,'/Users/johnf/Downloads/raw_low_res_dssg')

In [ ]:
out = pd.cut(nga_df.deprived_sev,5,duplicates='drop',labels=False)
out.plot.hist()
plt.show()

In [ ]:
# absolute path to search for all tiff files inside a specified folder
path = r"/Users/johnf/Downloads/raw_low_res_dssg/*.tif"
tif_files = glob.glob(path)
print(tif_files)


## Add satellite info to df

In [ ]:
for i, fname in enumerate(tif_files):
    title = Path(fname).name.lstrip("cpi").rstrip(".tif")
    print(f"Working with {title}: {i+1}/{len(tif_files)}...")
    # Convert to dataframe
    tmp = pg.geotiff_to_df(fname)
    print("Converted to dataframe!")
    print("Dataframe info:")
    print(tmp.info())
    print("Adding hex info...")
    tmp["hex_code"] = tmp[["latitude", "longitude"]].swifter.apply(
        lambda row: h3.geo_to_h3(row["latitude"], row["longitude"], 7), axis=1
    )
    tmp.drop(columns=["latitude", "longitude"], inplace=True)
    print("Done!")
    print("Summing within cells...")
    tmp = tmp.groupby(by=["hex_code"], as_index=False).sum()
    print("Joining to survey data...")
    # Aggregate ground truth to hexagonal cells with mean
    # NB automatically excludes missing data for households,
    # so differing quantities of data for different values
    nga_df = nga_df.merge(
        tmp.groupby(by=["hex_code"], as_index=False).sum(), how="left", on="hex_code"
    )
    print("Done!")


## Add OSM data

In [ ]:
osm_nga = pd.read_csv(
    "/Users/johnf/Downloads/raw_low_res_dssg/clean/nga_hex_osm.csv", sep="\t"
)
osm_nga.hex_id = osm_nga.hex_id.swifter.apply(h3.string_to_h3)
osm_nga.rename(columns={"hex_id": "hex_code"}, inplace=True)
osm_nga.drop(columns=["geometry"], inplace=True)


In [ ]:
nga_df = nga_df.merge(
    osm_nga.groupby(by=["hex_code"], as_index=False).mean(), how="left", on="hex_code"
)


In [ ]:
nga_df.to_csv("/Users/johnf/Downloads/raw_low_res_dssg/clean/nga_v2.csv", index=False)


### NOTE: used mean aggregation when lowering resolution, so for population data, the resulting estimates should be 100 times larger (100m -> 1km means 100x reduction)

# Extract centered images from tiffs at each point

In [ ]:
from stc_unicef_cpi.data.process_geotiff import extract_image_at_coords as ex_im


In [ ]:
nga_df.head()


In [ ]:
import rasterio
import matplotlib.pyplot as plt

lat, long = h3.h3_to_geo(609534210041970687)
all_ims = None
with rasterio.open(
    "/Users/johnf/Downloads/raw_low_res_dssg/cpiSlopeData.tif", masked=True
) as open_file:
    windowed_im = ex_im(open_file, lat, long, 256, 256)
    print(np.isnan(windowed_im).sum() / np.prod(windowed_im.shape))
    # plt.imshow(windowed_im[0,:,:])
    # plt.show()
    if all_ims is None:
        all_ims = windowed_im
    else:
        all_ims = np.vstack((all_ims, windowed_im))


In [ ]:
from stc_unicef_cpi.data.process_geotiff import extract_ims_from_hex_codes as get_ims


In [ ]:
all_ims = get_ims(tif_files, nga_df.hex_code.values, 256, 256)


## Run below to save to disk - warning ~12GB sparse / ~17GB dense

In [ ]:
with open(
    "/Users/johnf/Downloads/raw_low_res_dssg/clean/all_bands_centered_v1.npy", "wb"
) as f:
    np.save(f, all_ims)


In [ ]:
from humanfriendly import format_size


def get_memsize(x):
    print("Memory size of array in bytes:", format_size(x.size * x.itemsize))


In [ ]:
from scipy.sparse import csr_matrix


In [ ]:
all_ims.shape


In [ ]:
sparse_ims = [[csr_matrix(band) for band in im] for im in all_ims]


In [ ]:
print(
    format_size(
        np.sum(
            [
                [
                    band.data.nbytes + band.indptr.nbytes + band.indices.nbytes
                    for band in im
                ]
                for im in sparse_ims
            ]
        )
    )
)


In [ ]:
import pickle
with "/Users/johnf/Downloads/raw_low_res_dssg/clean/sparse_all_bands_centered_v1.pkl", 'wb') as f:
    pickle.dump(sparse_ims,f)

In [ ]:
from stc_unicef_cpi.data.process_geotiff import (
    convert_tiffs_to_image_dataset as tiff_to_ims,
)


In [ ]:
nga_df = pd.read_csv("/Users/johnf/Downloads/raw_low_res_dssg/clean/nga.csv")


In [ ]:
all_ims = tiff_to_ims(
    "/Users/johnf/Downloads/raw_low_res_dssg/", nga_df.hex_code.values
)


In [ ]:
import rioxarray as rxr


def get_band_names(tiff_dir):
    all_files = glob.glob(tiff_dir + "/*.tif")
    band_names = []
    for tif_file in all_files:
        with rxr.open_rasterio(tif_file, masked=True) as open_file:
            tif_bands = open_file.attrs["long_name"]
            if type(tif_bands) == tuple:
                band_names.extend(tif_bands)
            elif type(tif_bands) == str:
                band_names.append(tif_bands)
            else:
                print(type(tif_bands))
    return band_names


all_bands = get_band_names("/Users/johnf/Downloads/raw_low_res_dssg")


In [ ]:
random_im = np.random.randint(0, len(all_ims))
random_band = np.random.randint(0, 25)
plt.imshow(all_ims[random_im, random_band])
plt.show()


In [ ]:
import rasterio


In [ ]:
from stc_unicef_cpi.data.process_geotiff import extract_image_at_coords

with rasterio.open(
    "/Users/johnf/Downloads/raw_low_res_dssg/cpiPopData.tif", masked=True
) as open_file:
    test_im = extract_image_at_coords(
        open_file, *nga_df.iloc[random_im][["LATNUM", "LONGNUM"]].values
    )
    pop_bands = open_file.descriptions


In [ ]:
# for name,band in zip(pop_bands,test_im):
#     fig,ax=plt.subplots(dpi=150)
#     ax.imshow(band)
#     ax.set_title(name)
# plt.show()


In [ ]:
for name,band in zip(all_bands,all_ims[random_im]): 
    fig,ax=plt.subplots(dpi=150)
    ax.imshow(band)
    ax.set_title(name) 
    ax.set_axis_off()
plt.show()
    

# HDX API testing

In [ ]:
from hdx.utilities.easy_logging import setup_logging
from hdx.api.configuration import Configuration
from hdx.data.dataset import Dataset

setup_logging()
try:
    Configuration.create(
        hdx_site="prod", user_agent="HighresChildPov", hdx_read_only=True
    )
except Exception:
    pass


In [ ]:
datasets = Dataset.search_in_hdx("nigeria", rows=10)
datasets = list(
    filter(
        lambda x: x.is_subnational()
        # x.is_requestable()
        and "nga" in list(map(lambda y: y.lower(), x.get_location_iso3s())),
        datasets,
    )
)
print(len(datasets))
# print([dataset.get_fieldnames() for dataset in datasets])
resources = Dataset.get_all_resources(datasets)
# print(resources)
url, path = resources[0].download()
print("Resource URL %s downloaded to %s" % (url, path))


In [ ]:
len([resource.get_file_type() for resource in resources])


## GeoPandas quick work

In [ ]:
import geopandas as gpd


### Sound quality idea 
Data from [here](http://data.noise-planet.org/index.html)

In [ ]:
sound_areas_dir = (
    r"/Users/johnf/Downloads/raw_low_res_dssg/extras/nga_sound/*.areas.geojson"
)
sound_areas_files = glob.glob(sound_areas_dir)
for i, sfile in enumerate(sound_areas_files):
    if i == 0:
        sounds_df = gpd.read_file(sfile, driver="GeoJSON")
    else:
        sounds_df = sounds_df.append(gpd.read_file(sfile, driver="GeoJSON"))


In [ ]:
# only ~462 15m hex cells covered so not very useful
print(len(sounds_df))


# NetCDF files (GDP_PPP + HDI)

In [ ]:
import rasterio
import rasterio.mask
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))


In [ ]:
# NB looks like HDI only at v course level level
# HDI_1990_2015_v2.nc
# The dataset has a global extent at 5 arc-min resolution,
# and the annual data is available for each year over 1990-2015
# so 26 bands overall
# GDP_PPP_30arcsec_v3.nc
# GDP is given in 2011 international US dollars
# The data is derived from GDP per capita (PPP),
# which is multiplied by gridded population data from
# Global Human Settlement (GHS)
# Dataset has a global extent at 30 arc-second resolution
# for three time steps: 1990, 2000, and 2015
# bbox coords from https://www.alamy.com/area-of-nigeria-isolated-on-a-solid-background-in-a-georeferenced-bounding-box-main-regional-division-distance-scale-labels-colored-elevation-map-image368545266.html
# NB 30 arc seconds is just under 1km
nga_bbox = (
    2 + 40 / 60 + 6 / 3600,  # left deg,min,sec
    4 + 16 / 60 + 13 / 3600,  # bottom ...
    14 + 40 / 60 + 35 / 3600,  # right ...
    13 + 53 / 60 + 31 / 3600,  # top ...
)
nga_shp = world[world.name == "Nigeria"].geometry
for ds in ["HDI_1990_2015_v2.nc", "GDP_PPP_30arcsec_v3.nc"]:
    with rasterio.open(
        f"netcdf:/Users/johnf/Downloads/raw_low_res_dssg/extras/{ds}", "r", masked=True
    ) as netf:
        # print(netf.read().shape)
        # show((netf,1),cmap='viridis')
        print(netf.res)  # shows pixel scale in crs units
        print(netf.window(*nga_bbox))
        nga_subset = netf.read(window=netf.window(*nga_bbox))
        print(nga_subset[-1].min(), nga_subset[-1].max())
        plt.imshow(np.log(nga_subset[-1, :, :] + 10), cmap="PiYG")
        plt.show()
        out_image, out_transform = rasterio.mask.mask(netf, nga_shp, crop=True)
        plt.imshow(np.log(out_image[-1, :, :] + 10), cmap="PiYG")
        plt.show()


In [ ]:
from stc_unicef_cpi.data.process_netcdf import netcdf_to_clipped_array


In [ ]:
import rasterio
with rasterio.open("netcdf:/Users/johnf/Downloads/raw_low_res_dssg/extras/GDP_PPP_30arcsec_v3.nc", "r", masked=True) as netf:
    print(netf.crs)

In [ ]:
for ds in ["HDI_1990_2015_v2.nc", "GDP_PPP_30arcsec_v3.nc"]:
    netcdf_to_clipped_array(
        f"/Users/johnf/Downloads/raw_low_res_dssg/extras/{ds}",
        save_dir="/Users/johnf/Downloads/raw_low_res_dssg/extras/",
        ctry_name="Nigeria",
        plot=True,
    )


In [ ]:
from stc_unicef_cpi.data.process_geotiff import clip_tif_to_ctry

In [ ]:
clip_tif_to_ctry(
    "/Users/johnf/Downloads/raw_low_res_dssg/extras/updated electricity consumption/2019/EC2019.tif",
    save_dir="/Users/johnf/Downloads/raw_low_res_dssg/extras",
    ctry_name="Nigeria",
)


In [ ]:
clip_tif_to_ctry(
    "/Users/johnf/Downloads/raw_low_res_dssg/extras/updated real GDP/2019/2019GDP.tif",
    save_dir="/Users/johnf/Downloads/raw_low_res_dssg/extras",
    ctry_name="Nigeria",
)



In [ ]:
import rasterio
import matplotlib.pyplot as plt
with rasterio.open("/Users/johnf/Downloads/raw_low_res_dssg/extras/Nigeria_EC2019.tif",'r') as f: 
    array = f.read() 
    plt.imshow(array[-1]) 
    print(array.min(),array.max(),array.shape)
    plt.show()

In [ ]:
with rasterio.open("/Users/johnf/Downloads/raw_low_res_dssg/extras/updated electricity consumption/2019/EC2019.tif",'r') as f: 
    print(f.meta)
    print(f.res)
    print(f.bounds)
    print(f.width,f.height)


In [ ]:
import geopandas as gpd 
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))


In [ ]:
world[world.name == 'Nigeria'].geometry